In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

# --- КОНСТАНТЫ И ПАРАМЕТРЫ ---
TRANS_FILE = '../data/транзакции в Мобильном интернет Банкинге.csv'
BEHAV_FILE = '../data/поведенческие паттерны клиентов.csv'
ENCODING = 'cp1251'
DELIMITER = ';'
DATE_FORMAT_SHORT = '%Y-%m-%d' 

FINAL_FEATURES_PATH = '../data/processed/transactions_with_features.csv'
GROUPING_KEY = 'user_id'
TIME_WINDOWS = ['1h', '12h', '24h']

print("--- ЗАДАЧА 1: Загрузка и первичная очистка (День 1) ---")

# 1. Загрузка данных
try:
    df_trans = pd.read_csv(TRANS_FILE, encoding=ENCODING, sep=DELIMITER, header=1) 
    df_behav = pd.read_csv(BEHAV_FILE, encoding=ENCODING, sep=DELIMITER)
except FileNotFoundError:
    print("❌ КРИТИЧЕСКАЯ ОШИБКА: Файлы не найдены!")
    raise

# 2. Очистка и нормализация имен колонок
def clean_cols(df):
    df.columns = df.columns.astype(str).str.replace(' ', '_').str.lower().str.replace('"', '').str.strip()
    df.columns = df.columns.str.replace(r'[^\w]+', '_', regex=True).str.strip('_')
    return df

df_trans = clean_cols(df_trans)
df_behav = clean_cols(df_behav)

# 3. Очистка колонки суммы (amount)
df_trans = df_trans[df_trans['target'].astype(str) != 'nan'].copy() 
df_trans['amount'] = df_trans['amount'].astype(str).str.replace(r'[\.,]', '', regex=True).astype(float)


# 4. Преобразование дат и переименование
df_trans['timestamp'] = pd.to_datetime(df_trans['transdatetime'], errors='coerce')

df_behav.rename(columns={'дата_совершенной_транзакции': 'date_behav'}, inplace=True)
df_trans.rename(columns={'transdate': 'date_trans'}, inplace=True) 

df_behav['date_behav'] = pd.to_datetime(df_behav['date_behav'], format=DATE_FORMAT_SHORT, errors='coerce')
df_trans['date_trans'] = pd.to_datetime(df_trans['date_trans'], format=DATE_FORMAT_SHORT, errors='coerce')

# 5. Переименование ID клиента
df_behav.rename(columns={'уникальный_идентификатор_клиента': 'user_id'}, inplace=True)
df_trans.rename(columns={'cst_dim_id': 'user_id'}, inplace=True)


# Фикс: Приведение user_id к строковому типу
df_trans['user_id'] = df_trans['user_id'].astype(str)
df_behav['user_id'] = df_behav['user_id'].astype(str)


# --- ЗАДАЧА 2: Слияние и финальная очистка (День 1-2) ---

df_merged = pd.merge(
    df_trans, 
    df_behav, 
    left_on=['user_id', 'date_trans'], 
    right_on=['user_id', 'date_behav'], 
    how='left',
    suffixes=('_trans', '_behav')
)

initial_rows = df_merged.shape[0]
df = df_merged[df_merged['timestamp'].notna()].copy()
rows_dropped = initial_rows - df.shape[0]

if df.shape[0] == 0:
    print(f"❌ КРИТИЧЕСКАЯ ОШИБКА: Потеряны ВСЕ {initial_rows} строк при очистке временных меток. Проверьте исходный файл на наличие данных.")
    raise ValueError("DataFrame пуст после очистки временных меток.")

df.sort_values(by='timestamp', inplace=True)
df.drop(columns=['date_trans', 'transdatetime', 'date_behav', 'docno', 'direction'], inplace=True, errors='ignore')

df.fillna(0, inplace=True) 

print(f"✅ Таблицы объединены и очищены. Размер: {df.shape}. Удалено строк (из-за NaT): {rows_dropped}")


# --- ЗАДАЧА 23: Создать признаки по времени ---
print("\n--- ЗАДАЧА 23: Создание признаков по времени ---")

# Извлекаем временные признаки из timestamp
# Эти признаки помогают модели понять временные паттерны мошенничества
df['hour'] = df['timestamp'].dt.hour  # Час дня (0-23)
df['day_of_week'] = df['timestamp'].dt.dayofweek  # День недели (0=Понедельник, 6=Воскресенье)
df['day_of_month'] = df['timestamp'].dt.day  # День месяца (1-31)
df['month'] = df['timestamp'].dt.month  # Месяц (1-12)
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)  # Выходной день (1) или рабочий (0)
df['is_night'] = ((df['hour'] >= 22) | (df['hour'] < 6)).astype(int)  # Ночь (22:00-06:00)

# Циклическое кодирование для часа и дня недели (sin/cos) - помогает модели понять цикличность
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

print(f"✅ Создано 13 временных признаков: hour, day_of_week, day_of_month, month, is_weekend, is_night, и циклические кодирования")


# --- ЗАДАЧА 24: Создать признаки по пользователю (avg_amount, count) ---
print("\n--- ЗАДАЧА 24: Создание признаков по пользователю ---")

# Глобальные статистики по пользователю (вычисляются по всем его транзакциям до текущей)
# Эти признаки помогают понять типичное поведение пользователя

# Создаем копию для безопасных вычислений
df_sorted = df.sort_values(by=['user_id', 'timestamp']).copy()

# Признаки по пользователю (используем expanding window для накопления статистики)
df_sorted['user_tx_count_total'] = df_sorted.groupby(GROUPING_KEY).cumcount() + 1  # Общее количество транзакций пользователя до текущей
df_sorted['user_avg_amount_total'] = df_sorted.groupby(GROUPING_KEY)['amount'].expanding().mean().reset_index(level=0, drop=True)
df_sorted['user_std_amount_total'] = df_sorted.groupby(GROUPING_KEY)['amount'].expanding().std().reset_index(level=0, drop=True)
df_sorted['user_max_amount_total'] = df_sorted.groupby(GROUPING_KEY)['amount'].expanding().max().reset_index(level=0, drop=True)
df_sorted['user_min_amount_total'] = df_sorted.groupby(GROUPING_KEY)['amount'].expanding().min().reset_index(level=0, drop=True)

# Отклонение текущей транзакции от среднего пользователя
df_sorted['amount_diff_from_user_avg'] = df_sorted['amount'] - df_sorted['user_avg_amount_total']
df_sorted['amount_ratio_to_user_avg'] = df_sorted['amount'] / (df_sorted['user_avg_amount_total'] + 1e-8)  # +1e-8 чтобы избежать деления на 0

# Заполняем пропуски для первой транзакции каждого пользователя
df_sorted['user_std_amount_total'].fillna(0, inplace=True)
df_sorted['amount_diff_from_user_avg'].fillna(0, inplace=True)
df_sorted['amount_ratio_to_user_avg'].fillna(1, inplace=True)

# Обновляем основной датафрейм
df = df_sorted.copy()

print(f"✅ Создано 8 признаков по пользователю: tx_count, avg/std/max/min amount, отклонения от среднего")


# --- ЗАДАЧА 25: Создать rolling-window признаки ---
print("\n--- ЗАДАЧА 25: Создание rolling-window признаков ---")

# Rolling Window Features - скользящие окна по времени
# Эти признаки показывают активность пользователя в последние N часов
for window in TIME_WINDOWS:
    # 🔥 ФИНАЛЬНЫЙ ФИКС: Используем 'amount' для подсчета транзакций, так как 'user_id' может быть исключен из группы.
    df[f'tx_count_{window}'] = df.groupby(GROUPING_KEY).apply(
        lambda x: x.rolling(window=window, on='timestamp', closed='left')['amount'].count(),
        include_groups=False 
    ).reset_index(level=0, drop=True)
    
    df[f'tx_mean_amount_{window}'] = df.groupby(GROUPING_KEY).apply(
        lambda x: x.rolling(window=window, on='timestamp', closed='left')['amount'].mean(),
        include_groups=False 
    ).reset_index(level=0, drop=True)
    
    df[f'tx_std_amount_{window}'] = df.groupby(GROUPING_KEY).apply(
        lambda x: x.rolling(window=window, on='timestamp', closed='left')['amount'].std(),
        include_groups=False 
    ).reset_index(level=0, drop=True)
    
    # Дополнительные rolling признаки
    df[f'tx_sum_amount_{window}'] = df.groupby(GROUPING_KEY).apply(
        lambda x: x.rolling(window=window, on='timestamp', closed='left')['amount'].sum(),
        include_groups=False 
    ).reset_index(level=0, drop=True)
    
    df[f'tx_max_amount_{window}'] = df.groupby(GROUPING_KEY).apply(
        lambda x: x.rolling(window=window, on='timestamp', closed='left')['amount'].max(),
        include_groups=False 
    ).reset_index(level=0, drop=True)

print(f"✅ Создано {len(TIME_WINDOWS) * 5} rolling-window признаков для окон {TIME_WINDOWS}")


# --- ЗАДАЧА 26: Создать статистические признаки ---
print("\n--- ЗАДАЧА 26: Создание статистических признаков ---")

# Z-score признаков (отклонение от среднего в единицах стандартного отклонения)
# Z-score помогает выявить аномалии

# Z-score для amount относительно глобального распределения
global_amount_mean = df['amount'].mean()
global_amount_std = df['amount'].std() + 1e-8
df['amount_zscore'] = (df['amount'] - global_amount_mean) / global_amount_std

# Z-score для amount относительно пользователя (используем уже вычисленные статистики)
df['amount_zscore_user'] = df['amount_diff_from_user_avg'] / (df['user_std_amount_total'] + 1e-8)

# Процентили amount
df['amount_percentile'] = df['amount'].rank(pct=True)

# Отношение текущей суммы к максимальной/минимальной в окне
for window in TIME_WINDOWS:
    mean_col = f'tx_mean_amount_{window}'
    max_col = f'tx_max_amount_{window}'
    if mean_col in df.columns and max_col in df.columns:
        df[f'amount_ratio_to_max_{window}'] = df['amount'] / (df[max_col] + 1e-8)
        df[f'amount_ratio_to_mean_{window}'] = df['amount'] / (df[mean_col] + 1e-8)

# Коэффициент вариации (std/mean) для rolling окон
for window in TIME_WINDOWS:
    mean_col = f'tx_mean_amount_{window}'
    std_col = f'tx_std_amount_{window}'
    if mean_col in df.columns and std_col in df.columns:
        df[f'cv_amount_{window}'] = df[std_col] / (df[mean_col] + 1e-8)  # Coefficient of Variation

print(f"✅ Создано статистических признаков: z-scores, процентили, коэффициенты вариации")


# --- ЗАДАЧА 27: Создать признаки device-based & geo-based ---
print("\n--- ЗАДАЧА 27: Создание device-based & geo-based признаков ---")

# Поиск колонок, связанных с устройствами
device_cols = []
for col in df.columns:
    col_lower = col.lower()
    if 'phone' in col_lower or 'model' in col_lower or 'device' in col_lower or 'телефон' in col_lower or 'устройство' in col_lower:
        device_cols.append(col)

os_cols = []
for col in df.columns:
    col_lower = col.lower()
    if 'os' in col_lower or 'версия' in col_lower:
        os_cols.append(col)

# Поиск колонок с IP/геолокацией
ip_cols = []
geo_cols = []
for col in df.columns:
    col_lower = col.lower()
    if 'ip' in col_lower or 'адрес' in col_lower:
        ip_cols.append(col)
    if 'geo' in col_lower or 'location' in col_lower or 'страна' in col_lower or 'город' in col_lower:
        geo_cols.append(col)

print(f"Найдено колонок: устройств={len(device_cols)}, OS={len(os_cols)}, IP={len(ip_cols)}, геолокация={len(geo_cols)}")

# Device-based признаки
if len(device_cols) > 0:
    device_col = device_cols[0]
    # Частота использования устройства (frequency encoding)
    device_counts = df[device_col].value_counts()
    df['device_freq'] = df[device_col].map(device_counts) / len(df)
    
    # Количество уникальных устройств на пользователя
    df['user_unique_devices'] = df.groupby(GROUPING_KEY)[device_col].transform('nunique')
    
    # Изменение устройства (1 если устройство изменилось с предыдущей транзакции)
    df['device_changed'] = (df.groupby(GROUPING_KEY)[device_col].shift(1) != df[device_col]).astype(int)
    df['device_changed'].fillna(0, inplace=True)
    
    print(f"✅ Создано 3 device-based признака из колонки {device_col}")

# OS-based признаки
if len(os_cols) > 0:
    os_col = os_cols[0]
    # Частота использования OS версии
    os_counts = df[os_col].value_counts()
    df['os_freq'] = df[os_col].map(os_counts) / len(df)
    
    # Изменение OS версии
    df['os_changed'] = (df.groupby(GROUPING_KEY)[os_col].shift(1) != df[os_col]).astype(int)
    df['os_changed'].fillna(0, inplace=True)
    
    print(f"✅ Создано 2 OS-based признака из колонки {os_col}")

# IP-based признаки (если есть)
if len(ip_cols) > 0:
    ip_col = ip_cols[0]
    # Частота использования IP
    ip_counts = df[ip_col].value_counts()
    df['ip_freq'] = df[ip_col].map(ip_counts) / len(df)
    
    # Количество уникальных IP на пользователя
    df['user_unique_ips'] = df.groupby(GROUPING_KEY)[ip_col].transform('nunique')
    
    # Изменение IP адреса
    df['ip_changed'] = (df.groupby(GROUPING_KEY)[ip_col].shift(1) != df[ip_col]).astype(int)
    df['ip_changed'].fillna(0, inplace=True)
    
    print(f"✅ Создано 3 IP-based признака из колонки {ip_col}")
else:
    print("⚠️ IP адреса не найдены в данных")

# Geo-based признаки (если есть)
if len(geo_cols) > 0:
    geo_col = geo_cols[0]
    # Частота использования локации
    geo_counts = df[geo_col].value_counts()
    df['geo_freq'] = df[geo_col].map(geo_counts) / len(df)
    
    # Изменение локации
    df['geo_changed'] = (df.groupby(GROUPING_KEY)[geo_col].shift(1) != df[geo_col]).astype(int)
    df['geo_changed'].fillna(0, inplace=True)
    
    print(f"✅ Создано 2 geo-based признака из колонки {geo_col}")
else:
    print("⚠️ Геолокационные данные не найдены в данных")


# --- ЗАДАЧА 28: Создать признаки поведения (скорость операций) ---
print("\n--- ЗАДАЧА 28: Создание признаков поведения (скорость операций) ---")

# Lag Feature (time_since_last_tx) - уже было, но добавим дополнительные
df['time_since_last_tx'] = df.groupby(GROUPING_KEY)['timestamp'].diff().dt.total_seconds()
df['time_since_last_tx'].fillna(0, inplace=True)

# Время до следующей транзакции (forward-looking, но используем для анализа паттернов)
df['time_until_next_tx'] = df.groupby(GROUPING_KEY)['timestamp'].diff(-1).dt.total_seconds() * -1
df['time_until_next_tx'].fillna(0, inplace=True)

# Скорость транзакций (количество транзакций в час)
for window in TIME_WINDOWS:
    count_col = f'tx_count_{window}'
    if count_col in df.columns:
        # Преобразуем окно в часы для расчета скорости
        hours_map = {'1h': 1, '12h': 12, '24h': 24}
        hours = hours_map.get(window, 1)
        df[f'tx_rate_per_hour_{window}'] = df[count_col] / hours

# Средний интервал между транзакциями пользователя (rolling)
for window in TIME_WINDOWS:
    hours_map = {'1h': 1, '12h': 12, '24h': 24}
    hours = hours_map.get(window, 1)
    # Средний интервал = окно / количество транзакций
    count_col = f'tx_count_{window}'
    if count_col in df.columns:
        df[f'avg_interval_{window}'] = (hours * 3600) / (df[count_col] + 1)  # в секундах

# Аномалии в скорости (слишком быстро или слишком медленно)
# Если интервал между транзакциями очень маленький - подозрительно
df['is_rapid_tx'] = (df['time_since_last_tx'] < 60).astype(int)  # Меньше минуты между транзакциями

# Если интервал очень большой - тоже может быть подозрительно (необычное поведение)
user_avg_interval = df.groupby(GROUPING_KEY)['time_since_last_tx'].transform('mean')
df['interval_anomaly'] = (df['time_since_last_tx'] > user_avg_interval * 3).astype(int)  # В 3 раза больше среднего

# Частота транзакций в разные периоды дня
df['tx_in_morning'] = ((df['hour'] >= 6) & (df['hour'] < 12)).astype(int)
df['tx_in_afternoon'] = ((df['hour'] >= 12) & (df['hour'] < 18)).astype(int)
df['tx_in_evening'] = ((df['hour'] >= 18) & (df['hour'] < 22)).astype(int)

print(f"✅ Создано признаков поведения: временные интервалы, скорость транзакций, аномалии скорости")


# --- Дополнительно: Frequency Encoding для категориальных признаков ---
print("\n--- Дополнительно: Frequency Encoding для категориальных признаков ---")

# 3. Frequency Encoding для 'recipient_id' (если есть)
try:
    recipient_col = [col for col in df.columns if 'получателя' in col or 'recipient' in col.lower()][0]
    df[recipient_col].fillna('UNKNOWN_RECIPIENT', inplace=True)
    recipient_counts = df[recipient_col].value_counts()
    df['recipient_freq_encoding'] = df[recipient_col].map(recipient_counts) / len(df)
    df.drop(columns=[recipient_col], inplace=True, errors='ignore')
    print(f"✅ Создан frequency encoding для {recipient_col}")
except (IndexError, KeyError):
    print("⚠️ Предупреждение: Колонка 'Получатель' не найдена. Создание признака recipient_freq_encoding пропущено.")
    df['recipient_freq_encoding'] = 0


# --- Масштабирование числовых признаков ---
print("\n--- Масштабирование числовых признаков ---")

# Масштабирование числовых признаков (кроме временных циклических и бинарных)
cols_to_match = [
    'amount', 'time_since_last_tx', 'time_until_next_tx', 'recipient_freq_encoding',
    'user_avg_amount_total', 'user_std_amount_total', 'user_max_amount_total', 'user_min_amount_total',
    'amount_zscore', 'amount_zscore_user', 'amount_percentile',
    'device_freq', 'os_freq', 'ip_freq', 'geo_freq',
    'tx_rate_per_hour', 'avg_interval'
]

# Собираем все признаки для масштабирования
features_to_scale = []
for col in df.columns:
    # Исключаем временные циклические признаки (sin/cos), бинарные признаки, и target
    if any(match in col.lower() for match in ['_sin', '_cos', 'is_', 'changed', 'target', 'user_id', 'timestamp']):
        continue
    # Включаем числовые признаки
    if df[col].dtype in [np.int64, np.int32, np.float64, np.float32]:
        if col not in ['hour', 'day_of_week', 'day_of_month', 'month']:  # Исходные временные признаки не масштабируем
            features_to_scale.append(col)

# Добавляем rolling window признаки
for window in TIME_WINDOWS:
    for suffix in ['count', 'mean', 'std', 'sum', 'max']:
        col = f'tx_{suffix}_amount_{window}'
        if col in df.columns:
            features_to_scale.append(col)
    for suffix in ['rate_per_hour', 'avg_interval']:
        col = f'tx_{suffix}_{window}'
        if col in df.columns:
            features_to_scale.append(col)

# Добавляем статистические признаки
stat_cols = [col for col in df.columns if any(x in col for x in ['zscore', 'percentile', 'ratio', 'cv_amount'])]
features_to_scale.extend(stat_cols)

# Убираем дубликаты и проверяем наличие
features_to_scale = list(set([col for col in features_to_scale if col in df.columns]))

# Исключаем target если он есть
if 'target' in features_to_scale:
    features_to_scale.remove('target')

print(f"Найдено {len(features_to_scale)} признаков для масштабирования")

if len(features_to_scale) > 0:
    scaler = MinMaxScaler()
    df_features = df[features_to_scale].copy()
    df[features_to_scale] = scaler.fit_transform(df_features)
    print(f"✅ Масштабировано {len(features_to_scale)} признаков.")
else:
    print("⚠️ Признаки для масштабирования не найдены")


# --- ФИНАЛЬНОЕ СОХРАНЕНИЕ ---
print("\n--- Финальное сохранение обогащенного датасета ---")

# Заполняем оставшиеся пропуски
df.fillna(0, inplace=True)

# Удаляем временные колонки, которые не нужны для модели (оставляем только признаки)
cols_to_drop = ['timestamp']  # timestamp больше не нужен, все временные признаки извлечены
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

output_dir = os.path.dirname(FINAL_FEATURES_PATH)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

df.to_csv(FINAL_FEATURES_PATH, index=False)

print(f"\n✅ ФИНАЛЬНЫЙ ОБОГАЩЕННЫЙ ДАТАСЕТ ГОТОВ: {FINAL_FEATURES_PATH}")
print(f"Финальный размер таблицы с признаками: {df.shape}")
print(f"Количество признаков (без target): {len([c for c in df.columns if c != 'target'])}")

# Сводка по созданным признакам
print(f"\n📊 СВОДКА ПО СОЗДАННЫМ ПРИЗНАКАМ:")
print(f"  - Временные признаки: ~13 (час, день, циклические кодирования)")
print(f"  - Признаки по пользователю: ~8 (статистики по пользователю)")
print(f"  - Rolling-window признаки: ~{len(TIME_WINDOWS) * 5} (для окон {TIME_WINDOWS})")
print(f"  - Статистические признаки: ~{len([c for c in df.columns if 'zscore' in c or 'percentile' in c or 'ratio' in c or 'cv' in c])}")
print(f"  - Device/OS/IP/Geo признаки: зависит от наличия данных")
print(f"  - Признаки поведения: ~10+ (скорость, интервалы, аномалии)")
print(f"\n✅ Все задачи Feature Engineering (23-28) выполнены!")

--- ЗАДАЧА 1: Загрузка и первичная очистка (День 1) ---
✅ Таблицы объединены и очищены. Размер: (13155, 21). Удалено строк (из-за NaT): 0

--- ЗАДАЧА 23: Создание признаков по времени ---
✅ Создано 13 временных признаков: hour, day_of_week, day_of_month, month, is_weekend, is_night, и циклические кодирования

--- ЗАДАЧА 24: Создание признаков по пользователю ---
✅ Создано 8 признаков по пользователю: tx_count, avg/std/max/min amount, отклонения от среднего

--- ЗАДАЧА 25: Создание rolling-window признаков ---
✅ Создано 15 rolling-window признаков для окон ['1h', '12h', '24h']

--- ЗАДАЧА 26: Создание статистических признаков ---
✅ Создано статистических признаков: z-scores, процентили, коэффициенты вариации

--- ЗАДАЧА 27: Создание device-based & geo-based признаков ---
Найдено колонок: устройств=2, OS=5, IP=0, геолокация=0
✅ Создано 3 device-based признака из колонки количество_разных_моделей_телефона__phone_model__за_последние_30_дней___насколько_часто_клиент__менял_устройство__по_ло